#### En este notebook me enfoco en productos de alimentos congelados:
- Análisis exploratorio de datos.
- Clusterización RFV utilizando Kmeans.
- Market Basket con algoritmo A priori.

In [ ]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import datetime as dt
import numpy as np
import sidetable as stb

from mlxtend.frequent_patterns import apriori, association_rules

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist


pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

In [ ]:
#!conda install -c conda-forge yellowbrick
#from yellowbrick.cluster import KElbowVisualizer


In [ ]:
#yconda install -c conda-forge sidetable
#conda install -c districtdatalabs yellowbrick
#conda install -c sklearn.utils
#yellowbrick.download()

SELECT s.SaleId, s.SaleDate, sc.CustomerId, s.ProductId, s.ProductName, s.Amount,s.ProductWeightGrams * 0.001 'Kilos', s.BranchOfficeId, s.PromotionId, s.PromotionName
FROM dbo.Sale s
INNER JOIN dbo.SaleCustomer sc ON sc.SaleId=s.SaleId
WHERE s.SaleId IN
    (
    SELECT sbis.SaleId FROM dbo.Sale sbis
    INNER JOIN [dbo].[powerbi_franquicias_ventas_articulos] prod ON prod.codigo = sbis.ProductId
    WHERE prod.ptipo = 'H'
    AND prod.idCategoria = 3
    GROUP BY sbis.SaleId
    )
AND s.InvalidatedDate IS NULL
AND s.SaleDate > '2019-04-30'

In [ ]:
productos= pd.read_csv(r'C:\Users\Pablo\OneDrive - Helacor S.A\Club Grido\Informes de analítica\Alimentos congelados\productos.csv')
productos.drop_duplicates(subset=['codigo'],inplace=True)
productos.head()

## Importo y analizo ventas

Ventas a partir del 1/5/2021

In [ ]:
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})

In [ ]:
ventas= pd.read_csv(r'C:\Users\Pablo\OneDrive - Helacor S.A\Club Grido\Informes de analítica\Alimentos congelados\ventas_congelados.csv')
ventas.head()

In [ ]:
ventas.dropna(subset=['ProductId'],inplace=True)

In [ ]:
ventas= ventas.merge(productos[['codigo','linea_producto','idCategoria']], left_on='ProductId', right_on='codigo',how='left')
ventas.shape

In [ ]:
ventas.isnull().sum()

In [ ]:
ventas_alimentos = ventas[ventas.idCategoria ==3]

##### Analizo evolución de ventas

In [ ]:
ventas['mes'] = pd.to_datetime(ventas['SaleDate']).dt.month
ventas['año'] = pd.to_datetime(ventas['SaleDate']).dt.year

In [ ]:
X= ventas[ventas.idCategoria ==3].groupby(["año","mes"],as_index=False)["Kilos"].sum()
X.columns= ['año','mes','Kilos']
X["mes-año"] = X["mes"].astype(str)+ "-" + X["año"].astype(str)

fig, ax = plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', labelrotation = 30)
fig = sns.lineplot(data= X, x= "mes-año",y= "Kilos")
#fig.show()

Las ventas no se recuperaron luego del cierre por pandemia

In [ ]:
print('El ', round(100 - ventas_alimentos.PromotionId.isnull().sum()/ventas_alimentos.shape[0]*100,1),' % de las ventas fueron gracias a promociones')

In [ ]:
# Participación de las promociones

kilos_totales_promo= ventas_alimentos[~ventas_alimentos.PromotionId.isnull()].Kilos.sum()
ventas_promos = pd.DataFrame(ventas_alimentos.groupby('PromotionName').Kilos.sum()).reset_index()
ventas_promos['% kilos'] = round(ventas_promos.Kilos / kilos_totales_promo*100,1)
ventas_promos.sort_values('% kilos',ascending=False,inplace=True)
ventas_promos.head(5)

Las promos más exitosas son puramente de Frizzio.

In [ ]:
# Participación de los productos

kilos_totales_alimento= ventas_alimentos.Kilos.sum()
ventas_prod = pd.DataFrame(ventas_alimentos.groupby('ProductName').Kilos.sum()).reset_index()
ventas_prod['% kilos'] = round(ventas_prod.Kilos / kilos_totales_alimento*100,1)
ventas_prod.sort_values('% kilos',ascending=False,inplace=True)
ventas_prod.head(5)

In [ ]:
# Participación de las lineas de producto

ventas_linea = pd.DataFrame(ventas_alimentos.groupby('linea_producto').Kilos.sum()).reset_index()
ventas_linea['% kilos'] = round(ventas_linea.Kilos / kilos_totales_alimento*100,1)
ventas_linea.sort_values('% kilos',ascending=False,inplace=True)
ventas_linea.head(5)

In [ ]:
#Participacion por linea esta temporada

kilos_temp_alim= ventas_alimentos[pd.to_datetime(ventas.SaleDate).dt.date >= dt.datetime(2021,5,1).date()].Kilos.sum()
ventas_linea = pd.DataFrame(ventas_alimentos[pd.to_datetime(ventas.SaleDate).dt.date >= dt.datetime(2021,5,1).date()]\
                            .groupby('linea_producto').Kilos.sum()).reset_index()
ventas_linea['% kilos'] = round(ventas_linea.Kilos / kilos_temp_alim*100,1)
ventas_linea.sort_values('% kilos',ascending=False,inplace=True)
ventas_linea.head(5)

Analizo la evolucion de las lineas principales post pandemia

In [ ]:
lineas_top=['Pizzas', 'Bastoncitos', 'Pechuguitas','Empanadas','Papas']
X = ventas[ventas.linea_producto.isin(lineas_top)].groupby(["año","mes",'linea_producto'],as_index=False)["Kilos"].sum()
X['mes-año'] = X['año'].astype('str') + '-' + X['mes'].astype('str')

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', labelrotation = 30)
fig = sns.lineplot(data=X,x='mes-año',y='Kilos',hue='linea_producto')

In [ ]:
lineas_top=['Pizzas', 'Bastoncitos', 'Pechuguitas','Empanadas','Papas']
X = ventas[(ventas.linea_producto.isin(lineas_top)) & (pd.to_datetime(ventas.SaleDate).dt.date >= dt.datetime(2020,4,1).date())]\
    .groupby(["año","mes",'linea_producto'],as_index=False)["Kilos"].sum()
X['mes-año'] = X['año'].astype('str') + '-' + X['mes'].astype('str')

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', labelrotation = 30)
fig = sns.lineplot(data=X,x='mes-año',y='Kilos',hue='linea_producto')

In [ ]:
lineas_top=['Bastoncitos', 'Pechuguitas','Empanadas','Papas']
X = ventas[(ventas.linea_producto.isin(lineas_top)) & (pd.to_datetime(ventas.SaleDate).dt.date >= dt.datetime(2020,4,1).date())]\
    .groupby(["año","mes",'linea_producto'],as_index=False)["Kilos"].sum()
X['mes-año'] = X['año'].astype('str') + '-' + X['mes'].astype('str')

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', labelrotation = 30)
fig = sns.lineplot(data=X,x='mes-año',y='Kilos',hue='linea_producto')

La unica linea que volvió a crecer consistentemente fue la de pizzas, el resto aleatorio

##### Analizo días de ventas

In [ ]:
import locale
import calendar
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')
dia = pd.to_datetime(ventas_alimentos['SaleDate']).dt.weekday.apply(lambda x: calendar.day_name[x])

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.tick_params(axis='x', labelrotation = 30)
#plt.title('Compras por día de la semana')
fig = sns.countplot(y=dia, order = list(calendar.day_name), color='grey')
ax.set_xlabel('',loc='center')
ax.set_ylabel('',loc='center')

In [ ]:
(dia.value_counts(normalize=True)*100).apply(lambda x: round(x,1))

##### Analizo las horas de mayor venta

In [ ]:
horas = pd.to_datetime(ventas_alimentos['SaleDate']).dt.hour

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.tick_params(axis='x', labelrotation = 30)
plt.title('')
fig = sns.countplot(y=horas, palette=["#0C3B65"])
ax.set_xlabel('',loc='center')
ax.set_ylabel('',loc='center')

In [ ]:
dia_hora = pd.DataFrame(data={'dia':dia,'horas':horas})
dia_hora.head()

In [ ]:
fig, axes = plt.subplots(nrows=7,ncols= 1,figsize=(10,40))

for i,n_dia in enumerate(calendar.day_name[:]):
    axes[i].tick_params(axis='x', labelrotation = 30)
    axes[i].title.set_text(n_dia)
    sns.countplot(y=dia_hora[dia_hora.dia == n_dia].horas, palette=["#0C3B65"],ax=axes[i])

In [ ]:
ventas_alimentos.SaleDate.max()

### Análisis de clientes

##### Agrupo por clientes en ventas de alimentos

In [ ]:
lineas_top=['Pizzas','Bastoncitos', 'Pechuguitas','Empanadas','Papas']
ventas_alimentos['linea_producto']= np.where(~ventas_alimentos['linea_producto'].isin(lineas_top),'otras lineas',ventas_alimentos['linea_producto'])


clientes_alimentos = ventas_alimentos.groupby('CustomerId').agg({'Kilos':'sum','SaleId':'nunique','SaleDate':'max','linea_producto':'nunique'}).reset_index()
clientes_alimentos.rename(columns={'Kilos':'Kilos alimentos','SaleId':'Cantidad compras alimentos','SaleDate':'ultima_compra','linea_producto':'lineas_alim_probadas'},inplace=True)
ultima_fecha = pd.to_datetime(ventas_alimentos.SaleDate).dt.date.max()
clientes_alimentos['recencia_alimentos'] = (ultima_fecha - pd.to_datetime(clientes_alimentos.ultima_compra).dt.date)/ np.timedelta64(1,'D')

In [ ]:
#clientes_alimentos_lineas = ventas_alimentos.groupby(['CustomerId','linea_producto']).Kilos.sum().unstack().fillna(0).reset_index().head(10)
#clientes_alimentos = clientes_alimentos.merge(clientes_alimentos_lineas,on='CustomerId',how='left')
clientes_alimentos.head()

In [ ]:
cliente_promo = pd.DataFrame(ventas_alimentos[~ventas_alimentos.PromotionId.isnull()].groupby('CustomerId').SaleId.nunique().reset_index())

cliente_promo.rename(columns={'SaleId':'cantidad_promos_alimentos'},inplace=True)
cliente_promo.head()

In [ ]:
clientes_alimentos.shape

In [ ]:
cliente_promo.drop_duplicates(subset=['CustomerId'],inplace=True)
clientes_alimentos.drop_duplicates(subset=['CustomerId'],inplace=True)

clientes_alimentos = clientes_alimentos.merge(cliente_promo,on='CustomerId',how='left')
clientes_alimentos.shape

##### Agrupo por clientes en todas las ventas

In [ ]:
clientes_total = ventas.groupby('CustomerId').agg({'Kilos':'sum','SaleId':'nunique','SaleDate':'max'}).reset_index()
clientes_total.columns= ['CustomerId', 'Kilos totales','Cantidad de compras','ultima_compra']

ultima_fecha = pd.to_datetime(ventas.SaleDate).dt.date.max()
clientes_total['recencia'] = (ultima_fecha - pd.to_datetime(clientes_total.ultima_compra).dt.date)/ np.timedelta64(1,'D')
clientes_total.head()

##### Consolido todas las variables agrupadas en una sola tabla de clientes

In [ ]:
consumo_clientes = clientes_alimentos.merge(clientes_total,on='CustomerId')
consumo_clientes.head()

In [ ]:
print('Hay', consumo_clientes.shape[0],' clientes que consumieron alimentos congelados desde el 01-05-2019')

In [ ]:
consumo_clientes['% Alimentos'] = round(consumo_clientes['Kilos alimentos']/consumo_clientes['Kilos totales']*100,0)

In [ ]:
consumo_clientes['% Alimentos'].describe()

In [ ]:
plt.hist(consumo_clientes['% Alimentos'],bins=100)
plt.show()

In [ ]:
clusters = [0,5.9,34.5,95.5,100]
nombres_clusters =["Muy poco","Poco","Mucho","Exclusivo alimentos"]
consumo_clientes['Consumo_alimentos'] = pd.cut(consumo_clientes['% Alimentos'],bins= clusters, labels = nombres_clusters)

In [ ]:
consumo_clientes.stb.freq(['Consumo_alimentos'])

Armar graficos de 2 variables pintando por cluster armado

##### Importo otros datos de clientes y uno tablas

Consulta SQL Utilizada:

SELECT c.CustomerId, c.BirthDate, c.GenderCode, c.MaritalStatus, c.ActivatedDate, seg.categoria
FROM dbo.Customer c
INNER JOIN dbo.clientes_segmentacion seg ON seg.CustomerId = c.CustomerId

In [ ]:
clientes = pd.read_csv(r'C:\Users\Pablo\OneDrive - Helacor S.A\Club Grido\Informes de analítica\Alimentos congelados\customers.csv')
clientes.head()

In [ ]:
clientes['edad'] = (dt.datetime.today() - pd.to_datetime(clientes.BirthDate)) / np.timedelta64(1,'Y')
clientes['edad'] = clientes['edad'].apply(np.floor).astype('int')

clientes['años de activo'] = (dt.datetime.today().date() - pd.to_datetime(clientes.ActivatedDate).dt.date) / np.timedelta64(1,'Y')

In [ ]:
clientes.drop(columns=['BirthDate','ActivatedDate','MaritalStatus'],inplace=True)

In [ ]:
clientes.drop_duplicates(subset=['CustomerId'],inplace=True)

In [ ]:
clientes.isnull().sum()/clientes.shape[0]*100

In [ ]:
consumo_clientes = consumo_clientes.merge(clientes, how='left', on='CustomerId')
consumo_clientes.head()

In [ ]:
consumo_clientes['Frecuencia de compra'] = round(365 * consumo_clientes['años de activo'] / consumo_clientes['Cantidad de compras'],0)
consumo_clientes['Frecuencia de compra alimento'] = round(365 * consumo_clientes['años de activo'] / consumo_clientes['Cantidad compras alimentos'],0)

In [ ]:
consumo_clientes.head()

In [ ]:
consumo_clientes.isnull().sum()

In [ ]:
consumo_clientes.shape

In [ ]:
consumo_clientes['Frecuencia de compra'].describe()

In [ ]:
consumo_clientes['Frecuencia de compra alimento'].describe()

##### Gráficas según segmento

In [ ]:
consumo_clientes.groupby('Consumo_alimentos').agg({'edad':['mean','median'],'% Alimentos':'mean','Kilos alimentos':['mean','median'],'Cantidad compras alimentos':'mean'})

In [ ]:
consumo_clientes.groupby(['Consumo_alimentos','GenderCode']).agg({'edad':['mean','median'],'% Alimentos':'mean','Kilos alimentos':['mean','median'],'Cantidad compras alimentos':'mean'})

Hay 2.664.536 socias del total de 4.375.681

Es decir, el 60% de los socios de Club Grido, son mujeres.

In [ ]:
consumo_clientes.groupby(['Consumo_alimentos','GenderCode']).agg({'CustomerId':'nunique'}).groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [ ]:
clientes.stb.freq(['GenderCode'])

In [ ]:
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]
pl = sns.scatterplot(data = consumo_clientes,x='edad', y="GenderCode",hue='Consumo_alimentos',size= "Kilos alimentos",palette= pal)
pl.set_title("Edad y género según segmento")
plt.legend()
plt.show()

# Clusterización

La idea es ir armando clusterización de clientes para obtener ideas
\
\
\
\
\
.

### RFV de alimentos

In [ ]:
consumo_clientes.columns

In [ ]:
X = consumo_clientes[['CustomerId','Kilos alimentos', 'Cantidad compras alimentos', 'recencia_alimentos']]

##### Analizar outliers y correlaciones

In [ ]:
#correlation matrix
corrmat= X.corr()
plt.figure(figsize=(20,20))  
sns.heatmap(corrmat,annot=True, center=0)

Analizo la existencia de outliers con gráficos pareados

In [ ]:
#To plot some selected features 
#Setting up colors prefrences
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]
cmap = colors.ListedColormap(["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])
#Plotting following features
plt.figure()
sns.pairplot(X,palette= (["#682F2F","#F3AB60"]))
#Taking hue 
plt.show()

A partir de los gráficos, puedo reconocer los outliers.
- Kilos alimentos tiene que ser < a 300


In [ ]:
X= X[X['Kilos alimentos']<300]

Elimino nulos

In [ ]:
X.isnull().sum()

In [ ]:
X.dropna(how='any',inplace=True)

##### Normalizar variables

Esto se hace para que las distancias sean las mismas en todos los ejes

In [ ]:
scaler = StandardScaler()
scaler.fit(X.drop(columns=['CustomerId']))
X_standard = pd.DataFrame(scaler.transform(X.drop(columns=['CustomerId'])),columns= X.drop(columns=['CustomerId']).columns)
X_standard.head()

##### Principal Component Analysis (PCA)

Tecnica para reducir la dimensionalidad

In [ ]:
pca = PCA(n_components=2)

In [ ]:
principalComponents = pca.fit_transform(X_standard)

In [ ]:
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
principalDf.head()

In [ ]:
pca.explained_variance_ratio_

Tecnica para reducir la dimensionalidad. Con 2 variables explican el 96% de los datos.

##### Método del codo para definir cantidad de clusters

In [ ]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 10)

for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(principalDf)
    kmeanModel.fit(principalDf)

    distortions.append(sum(np.min(cdist(principalDf, kmeanModel.cluster_centers_,'euclidean'), axis=1)) / principalDf.shape[0])
    inertias.append(kmeanModel.inertia_)

    mapping1[k] = sum(np.min(cdist(principalDf, kmeanModel.cluster_centers_,'euclidean'), axis=1)) / principalDf.shape[0]
    mapping2[k] = kmeanModel.inertia_


In [ ]:
for key, val in mapping1.items():
    print(f'{key} : {val}')

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [ ]:
plt.plot(K, inertias, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Inertias')
plt.title('The Elbow Method using Inertia')
plt.show()

Pruebo con k=3

#### Implemento el algoritmo

In [ ]:
kmeans = KMeans(n_clusters=3).fit(principalDf)
labels = kmeans.predict(principalDf)
# Obtengo los 3 centroides
C = kmeans.cluster_centers_

In [ ]:
X['labels_kmeans'] = labels
principalDf['labels_kmeans'] = labels
consumo_clientes = consumo_clientes.merge(X[['CustomerId','labels_kmeans']],on='CustomerId',how='left')
consumo_clientes.head()

##### Evalúo el algoritmo

In [ ]:
metrics.silhouette_score(principalDf, labels,sample_size=250000)

In [ ]:
principalDf.shape

In [ ]:
px = 1/plt.rcParams['figure.dpi']  # pixel in inches
plt.figure(figsize=(1280*px,769*px))
pl = sns.scatterplot(data = principalDf, x='principal component 1', y='principal component 2',hue='labels_kmeans')
pl.set_title("Customer Cluster's Profile Based on kg alimentos and kg totales")
plt.legend()

plt.show()

In [ ]:
X.stb.freq(['labels_kmeans'],cum_cols=False)

In [ ]:
sns.countplot(y=X['labels_kmeans'],palette = ['#880105','#EDC305','#88AD05'])
plt.ylabel('')

##### Interpreto el algoritmo

In [ ]:
#Setting up colors prefrences
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]
cmap = colors.ListedColormap(["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])

#Plotting following features

px = 1/plt.rcParams['figure.dpi']  # pixel in inches
plt.figure(figsize=(1280*px,769*px))

sns.pairplot(X,hue='labels_kmeans',palette =pallet[:3])

#Taking hue 
plt.show()

#### Interpretación

- Kilos alimentos: grupo 2 es el que más kg compra, 0 y 1 aproximadamente lo mismo y poco
- Cantidad de compras: grupo 2 es el que más compras hace, 0 y 1 muy pocas
- Recencia: grupo 2 se distribuye por todo el rango, 0 compró hace poco y 1 probó alguna vez pero no volvió a comprar.

##### Entonces:

* Grupo 0: Clientes nuevos de alimentos congelados, todavía tienen poco consumo pero siguen probando los productos.
* Grupo 1: Probaron los alimentos congelados pero no se engancharon con los mismos.
* Grupo 2: son los clientes oro de alimentos congelados, algunos están "dormidos"

El grupo 0 se convertirá en 1 o 2 dependiendo de su comportamiento inmediato.

In [ ]:
X['labels_kmeans'] = np.where(X['labels_kmeans']==0,'Clientes perdidos',np.where(X['labels_kmeans']==1,'Clientes top alimentos','Clientes activos alimentos'))

In [ ]:
plt.figure(figsize=(10,5))
pl = sns.scatterplot(data = X, x='recencia_alimentos', y='Kilos alimentos',hue='labels_kmeans',palette = ['#880105','#EDC305','#88AD05'])
pl.set_title("Grupos según recencia de alimentos y kilos de alimentos comprados")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
pl = sns.scatterplot(data = X, x='Cantidad compras alimentos', y='Kilos alimentos',hue='labels_kmeans',palette = ['#880105','#EDC305','#88AD05'])
pl.set_title("Grupos según recencia de alimentos y kilos de alimentos comprados")
plt.legend()
plt.show()

In [ ]:
consumo_clientes.columns

In [ ]:
consumo_clientes['% promociones'] = round(consumo_clientes['cantidad_promos_alimentos']/consumo_clientes['Cantidad compras alimentos']*100,1)

In [ ]:
consumo_clientes['labels_kmeans'] = np.where(consumo_clientes['labels_kmeans']==0,'Clientes perdidos',\
                        np.where(consumo_clientes['labels_kmeans']==1,'Clientes top alimentos','Clientes activos alimentos'))

In [ ]:
consumo_clientes.groupby('labels_kmeans').agg({'edad':['mean','median'],'Kilos alimentos':'mean','Cantidad compras alimentos':'mean','lineas_alim_probadas':'mean','recencia_alimentos':['mean','median'],'años de activo':['mean','median'],'% promociones':['mean','median']})

In [ ]:
consumo_clientes.groupby('labels_kmeans').agg({'Kilos totales':['mean','median'],'Cantidad de compras':['mean','median'],'recencia':['mean','median'],'Frecuencia de compra':['mean','median']})

In [ ]:
consumo_clientes.groupby(['labels_kmeans','GenderCode']).agg({'CustomerId':'nunique'}).groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [ ]:
consumo_clientes.groupby(['labels_kmeans','categoria']).agg({'CustomerId':'nunique'}).groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [ ]:
consumo_clientes.groupby(['labels_kmeans','GenderCode']).agg({'edad':['mean','median'],'años de activo':['mean','median']})

In [ ]:
consumo_clientes.groupby('labels_kmeans')['Kilos alimentos'].sum() / consumo_clientes['Kilos alimentos'].sum()*100

### A Priori Algorithm

Armo la columna línea con lo que me interesa

In [ ]:
ventas.head()

In [ ]:
lineas_top=['Pizzas','Bastoncitos', 'Pechuguitas','Empanadas','Papas']

ventas['Linea'] = ventas['linea_producto']

ventas['Linea'] = np.where(ventas['idCategoria']==3,'Congelado',ventas['Linea'])
ventas['Linea'] = np.where(ventas['linea_producto'].isin(lineas_top),ventas['linea_producto'],ventas['Linea'])

ventas['Linea'] = np.where((ventas['linea_producto'] == 'Palitos')|(ventas['linea_producto'] == 'Bombones') | \
                           (ventas['linea_producto'] == 'Tortas')|(ventas['linea_producto'] == 'Postres')\
                           |(ventas['linea_producto'] == 'Semifrío'),'Cajas',ventas['Linea'])
ventas['Linea'] = np.where(((~ventas['ProductName'].str.contains('X 8',case=False)) & (~ventas['ProductName'].str\
    .contains('X 10',case=False))&(~ventas['ProductName'].str.contains('X 20'))&(~ventas['ProductName'].str.contains('x20')))\
    & ((ventas['linea_producto'] == 'Palitos')|(ventas['linea_producto'] == 'Bombones')),'Por unidad',ventas['Linea'])


ventas['Linea'] = np.where(ventas['Linea']=='Factura','Café',ventas['Linea'])

ventas['Linea'] = np.where(ventas['Linea']=='Especial','Líneas Especiales',ventas['Linea'])

#ventas['Linea'] = np.where(ventas['Linea']=='Helado x Bocha','Por unidad',ventas['Linea'])

ventas['Linea'] = np.where(ventas['ProductName'].str.contains('1 Kilo',case=False),'1 kg',ventas['Linea'])

#ventas['Linea'] = np.where((ventas['Linea']=='Cups y vasos')|(ventas['Linea']=='Tops'),'A definir',ventas['Linea'])

# Se podria agregar helado por bocha + por unidades

In [ ]:
ventas = ventas[ventas.PromotionId.isnull()].copy()
ventas.drop(columns=['ProductId','BranchOfficeId','PromotionId','PromotionName','Kilos'],inplace=True)

In [ ]:
ventas.Linea.value_counts(normalize=True)*100

In [ ]:
ventas.head()

In [ ]:
#ventas.Linea.unique()

In [ ]:
lineas_elegidas = ['Cajas', '1 kg', 'Por unidad','Helado x Bocha', 'Familiar', 'Helado X Kilo',
       'Pote 1 Litro', 'Pizzas', 'Congelado', 'Pechuguitas','Bastoncitos', 'Café', 'Papas', 'Empanadas']
basket= ventas[ventas.Linea.isin(lineas_elegidas)].groupby(['SaleId', 'Linea'])['Amount'].sum().unstack().reset_index().fillna(0).set_index('SaleId')
basket.head()

In [ ]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

In [ ]:
basket = basket.applymap(hot_encode)
basket= basket.astype('int32')

In [ ]:
# Building the model
frq_items = apriori(basket, min_support = 0.0015, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules


- Support: numero de transacciones donde aparece x item sobre el total de transacciones
- Confidence: probabilidad condicional P(A|B) = (P(A y B)/P(A)): la prob de que se venda B dado que se vendió A
- Lift: probabilidad de que ambos items sean comprados juntos. 
        - Si lift < 1, hay mas probabilidad de que los items se compren por separado
        - Si lift =1, no hay asociacion entre los items
        - Si lift > 1, hay asociacion entre los items

In [ ]:
frq_items

Mismo análisis pero para esta temporada

In [ ]:
lineas_elegidas = ['Cajas', '1 kg', 'Por unidad','Helado x Bocha', 'Familiar', 'Helado X Kilo',
       'Pote 1 Litro', 'Pizzas', 'Congelado', 'Pechuguitas','Bastoncitos', 'Café', 'Papas', 'Empanadas']
basket= ventas[(ventas.Linea.isin(lineas_elegidas)) & (pd.to_datetime)].groupby(['SaleId', 'Linea'])['Amount'].sum().unstack().reset_index().fillna(0).set_index('SaleId')
basket.head()

In [ ]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

In [ ]:
# Building the model
frq_items = apriori(basket, min_support = 0.0015, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules


- Support: numero de transacciones donde aparece x item sobre el total de transacciones
- Confidence: probabilidad condicional P(A|B) = (P(A y B)/P(A)): la prob de que se venda B dado que se vendió A
- Lift: probabilidad de que ambos items sean comprados juntos. 
        - Si lift < 1, hay mas probabilidad de que los items se compren por separado
        - Si lift =1, no hay asociacion entre los items
        - Si lift > 1, hay asociacion entre los items

In [ ]:
basket = basket.applymap(hot_encode)
basket= basket.astype('int32')

In [ ]:
frq_items

### Analizo consumo de líneas por segmento

In [ ]:
consumo_clientes_linea = ventas.groupby(['CustomerId','Linea']).Kilos.sum().reset_index()
consumo_clientes_linea.head()

In [ ]:
consumo_clientes_linea = consumo_clientes_linea.merge(X[['CustomerId','labels_kmeans']],on='CustomerId')
consumo_clientes_linea.head()

In [ ]:
consumo_clientes_linea_grouped= consumo_clientes_linea.groupby(['labels_kmeans','Linea']).Kilos.sum().groupby(level=0).apply(lambda x:100 * x / float(x.sum())).reset_index()

lineas_grafico = ['1 kg', 'Bastoncitos', 'Cajas', 'Congelado', 'Empanadas','Familiar', 'Helado X Kilo', 'Helado x Bocha','Papas', 'Pechuguitas', 'Pizzas', 'Por unidad','Pote 1 Litro']
consumo_clientes_linea_grouped = consumo_clientes_linea_grouped[consumo_clientes_linea_grouped['Linea'].isin(lineas_grafico)]

consumo_clientes_linea_grouped['Linea'] = np.where(consumo_clientes_linea_grouped['Linea'] == 'Helado X Kilo', '1/4 y 1/2 kg', consumo_clientes_linea_grouped['Linea'])
consumo_clientes_linea_grouped['Linea'] = np.where(consumo_clientes_linea_grouped['Linea'] == 'Papas', 'Congelado', consumo_clientes_linea_grouped['Linea'])
consumo_clientes_linea_grouped['Linea'] = np.where(consumo_clientes_linea_grouped['Linea'] == 'Congelado', 'Otros congelados', consumo_clientes_linea_grouped['Linea'])
consumo_clientes_linea_grouped['Linea'] = np.where(consumo_clientes_linea_grouped['Linea'] == 'Cajas', 'Packs y tortas', consumo_clientes_linea_grouped['Linea'])
consumo_clientes_linea_grouped['Linea'] = np.where(consumo_clientes_linea_grouped['Linea'] == 'Helado x Bocha', 'Por unidad', consumo_clientes_linea_grouped['Linea'])

In [ ]:
plt.figure(figsize=(20,10))


fig, ax = plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', labelrotation = 30)
ax.grid(b=True,axis='y',color='black', linestyle='-', linewidth=0.2)
fig=sns.barplot(data=consumo_clientes_linea_grouped, x='Linea',y='Kilos',hue='labels_kmeans',palette = ['#EDC305','#880105','#88AD05'])
ax.set_xlabel('Línea de producto')
ax.set_ylabel('% Kilos consumidos sobre el total del segmento')



In [ ]:
plt.figure(figsize=(20,10))

fig, ax = plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', labelrotation = 30)
fig=sns.barplot(data=consumo_clientes_linea, x='Linea',y='Kilos',hue='labels_kmeans',palette = ['#880105','#EDC305','#88AD05'])

### Analizo localidades donde hubo ventas

##### Importo datos de franquicias y de localidades

In [ ]:
fqcias = pd.read_excel(r'C:\Users\Pablo\Helacor S.A\Equipo CLUB GRIDO - General\Datos\fqcias.xlsx')
fqcias.head()

In [ ]:
fqcias.drop_duplicates(subset=['BranchOfficeId'],inplace=True)

In [ ]:
localidades = pd.read_excel(r'C:\Users\Pablo\Helacor S.A\Equipo CLUB GRIDO - General\Datos\Localidades.xlsx')
localidades.head()

##### Saco la localidad donde el cliente realizó más compras

In [ ]:
customer_localidad = ventas_alimentos.groupby(['CustomerId','BranchOfficeId']).SaleId.nunique().reset_index()
customer_localidad.sort_values('CustomerId',inplace=True)
customer_localidad.sort_values('SaleId',ascending=False,inplace=True)
customer_localidad.drop_duplicates(subset=['CustomerId'],inplace=True)
customer_localidad = customer_localidad.merge(fqcias[['idLocalidad','BranchOfficeId']],on='BranchOfficeId')
customer_localidad.head()

In [ ]:
customer_localidad['idLocalidad'] = customer_localidad['idLocalidad'].astype('int')
customer_localidad = customer_localidad.merge(localidades[['Código','poblacion','Localidad']], left_on='idLocalidad', right_on='Código')
customer_localidad['tamaño_ciudad'] = np.where(customer_localidad.poblacion <20000,'Chica',np.where(customer_localidad.poblacion >150000,'Grande','Mediana'))
customer_localidad.drop(columns=['BranchOfficeId','SaleId','Código'],inplace=True)
customer_localidad.head()

##### Cruzo la localidad de cada cliente con sus datos

In [ ]:
consumo_clientes = consumo_clientes.merge(customer_localidad,on='CustomerId',how='left')
consumo_clientes.head()

In [ ]:
pd.DataFrame(consumo_clientes.groupby(['labels_kmeans','tamaño_ciudad']).CustomerId.nunique().groupby(level=0).apply(lambda x:100 * x / float(x.sum())))

In [ ]:
consumo_clientes.stb.freq(['labels_kmeans'])

In [ ]:
pd.DataFrame(consumo_clientes.groupby(['labels_kmeans','GenderCode','tamaño_ciudad']).CustomerId.nunique().groupby(level=0).apply(lambda x:100 * x / float(x.sum())))

##### Cruzo ventas con localidad y saco % alimentos en cada localidad

In [ ]:
ventas_localidad = ((ventas[['Kilos','BranchOfficeId']].merge(fqcias[['BranchOfficeId','idLocalidad','Localidad','Provincia']],on='BranchOfficeId',how='left')).groupby(['Localidad','Provincia']).Kilos.sum()).apply(lambda x: round(x,1)).reset_index()
ventas_localidad.shape

In [ ]:
ventas_alimentos = ventas_alimentos.merge(fqcias[['BranchOfficeId','idLocalidad','Localidad','Provincia']],on='BranchOfficeId',how='left')

In [ ]:
ventas_localidad_alim = (ventas_alimentos.groupby(['idLocalidad','Localidad','Provincia']).Kilos.sum()).apply(lambda x: round(x,1)).reset_index()
ventas_localidad_alim.rename(columns={'Kilos':'Kilos alimento'},inplace=True)
ventas_localidad_alim.head()

In [ ]:
ventas_localidad = ventas_localidad.merge(ventas_localidad_alim, on=['Localidad','Provincia'],how='left').fillna(0)
ventas_localidad.shape

In [ ]:
ventas_localidad['% alimento'] = round(ventas_localidad['Kilos alimento'] / ventas_localidad['Kilos'] * 100,1)
ventas_localidad.sort_values('% alimento',ascending=False,inplace=True)

#### Observo rdos de % alimentos

In [ ]:
ventas_localidad['% alimento'].describe()

In [ ]:
ventas_localidad[ventas_localidad.Kilos >20000]['% alimento'].describe()

In [ ]:
plt.hist(ventas_localidad['% alimento'],bins=100)
plt.ylabel('Cantidad de localidades')
plt.xlabel('Kg alimento / Kg totales x 100 (%)')
plt.title('Histograma de recuento de localidades')
plt.show()

In [ ]:
plt.hist(ventas_localidad[ventas_localidad.Kilos >20000]['% alimento'],bins=100)
plt.ylabel('Cantidad de localidades')
plt.xlabel('Kg alimento / Kg totales x 100 (%)')
plt.title('Histograma de recuento de localidades')
plt.show()

In [ ]:
ventas_localidad.sort_values('Kilos',ascending=False,inplace=True)
ventas_localidad.head(200)

In [ ]:
ventas_localidad['idLocalidad'] = ventas_localidad['idLocalidad'].astype('int')
ventas_localidad = ventas_localidad.merge(localidades[['Código','poblacion']], left_on='idLocalidad',right_on = 'Código')
ventas_localidad.shape

In [ ]:
ventas_localidad['% alimento'].describe()

In [ ]:
ventas_localidad['poblacion'].describe()

In [ ]:
ventas_localidad['tamaño_ciudad'] = np.where(ventas_localidad.poblacion <50000,'Chica',np.where(ventas_localidad.poblacion >1200000,'Grande','Mediana'))

In [ ]:
ventas_localidad.groupby(['tamaño_ciudad']).agg({'% alimento':['mean','median']})

In [ ]:
plt.hist(ventas_localidad[ventas_localidad['poblacion'] > 70000]['% alimento'],bins=100)
plt.show()
